In [2]:
!nvidia-smi

Mon Jun  5 23:41:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce MX150          WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P0               N/A /  N/A|      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
# !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate 
# !pip install transformers accelerate 

In [5]:
from transformers import pipeline , set_seed
from datasets import load_dataset , load_from_disk , load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM , AutoTokenizer, DataCollatorForSeq2Seq

import nltk
from nltk.tokenize import sent_tokenize

import torch
from tqdm import tqdm


nltk.download("punkt")

c:\Users\Mahmo\.conda\envs\text_summazrizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# !wget https://drive.google.com/file/d/1d1EJI_T3J8IX1Rz0irumuSL8vjQC6OS7/view?usp=share_link
# !wget https://drive.google.com/drive/folders/1fFSH4XAFZKFaLLeU3DoZLvlEhzAdT4PJ?usp=share_link

In [ ]:
# !ls "/content/drive/MyDrive"

'Colab Notebooks'	      EDA			  spamcollection
 current		      github_docker_help	  text_summarization
 CV_Mahmoud_Sayed_Qotb.docx   Mahmoud_Sayed_Qotb.mp4	  text_summ.ipynb
 CV_Mahmoud_Sayed_Qotb.pdf   'New folder (2)'
 CVs			      Robot-Arm-Gesture-Control


In [ ]:
# !cd "/content/drive/MyDrive"

In [ ]:
# !ls "/content/drive/MyDrive/text_summarization"

In [ ]:
# !unzip  "/content/drive/MyDrive/text_summarization/summarizer-data.zip"

Archive:  /content/drive/MyDrive/text_summarization/summarizer-data.zip
  inflating: samsum-test.csv         
  inflating: samsum-train.csv        
  inflating: samsum-validation.csv   
   creating: samsum_dataset/
 extracting: samsum_dataset/dataset_dict.json  
   creating: samsum_dataset/test/
  inflating: samsum_dataset/test/data-00000-of-00001.arrow  
  inflating: samsum_dataset/test/dataset_info.json  
  inflating: samsum_dataset/test/state.json  
   creating: samsum_dataset/train/
  inflating: samsum_dataset/train/data-00000-of-00001.arrow  
  inflating: samsum_dataset/train/dataset_info.json  
  inflating: samsum_dataset/train/state.json  
   creating: samsum_dataset/validation/
  inflating: samsum_dataset/validation/data-00000-of-00001.arrow  
  inflating: samsum_dataset/validation/dataset_info.json  
  inflating: samsum_dataset/validation/state.json  


In [11]:
from text_summarizer.constants import *

df_samsum = load_from_disk(PROJECT_DIR.joinpath("artifacts/data_ingestion/samsum_dataset"))
df_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
split_lens = [len (df_samsum[split]) for split in df_samsum]
print(f"Split lengths: {split_lens}")
print(f"Features: {df_samsum['train'].column_names}")
print('Ex:')
print("\nDialogue:")
print(df_samsum["test"][1]["dialogue"])
print("\nSummary")
print(df_samsum["test"][1]['summary'])



Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']
Ex:

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [13]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'],max_length=128,truncation=True)

  return {
      'input_ids':input_encodings['input_ids'] ,
      'attention_mask':input_encodings['attention_mask'] ,
      'labels': target_encodings['input_ids'] ,
  }

In [14]:
df_samsum_pt = df_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Mahmo\.conda\envs\text_summazrizer\lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
df_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [15]:
# Training
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [16]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16,
    optim="adamw_torch"
    )

In [17]:
trainer = Trainer(model=model_pegasus,args = trainer_args, 
                  tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                  
                  train_dataset=df_samsum_pt['test'],
                  eval_dataset=df_samsum_pt['validation'])

In [18]:
trainer.train()

  0%|          | 0/51 [00:00<?, ?it/s]You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.47 GiB already allocated; 0 bytes free; 3.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [22]:
# Evaluation
 
def generate_batch_sized_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[ i : i + batch_size]

def calculate_metric_on_test_ds(dataset,metric, model, tokenizer,
                                batch_size= 16, device= device,
                                column_text= "article",
                                column_summary= "highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary],batch_size))
  for article_batch, target_batch in tqdm(
      zip(article_batches,target_batches ), total=len(article_batches) ):
    
     
    inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                    padding="max_length", return_tensors="pt")
    
    summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                      attention_mask=inputs["attention_mask"].to(device), 
                      length_penalty=0.8, num_beams=8, max_length=128)
    ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
    decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
    metric.add_batch(predictions=decoded_summaries, references=target_batch)
     #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [23]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')


<ipython-input-23-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [24]:
score = calculate_metric_on_test_ds(
    df_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

  0%|          | 0/5 [00:02<?, ?it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.026444,0.0,0.026444,0.026444


In [25]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [26]:
#Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")


In [27]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text =df_samsum["test"][0]["dialogue"]

reference = df_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

## 
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
